In [2]:
from yohane.force_alignment import compute_alignments, load_audio
from yohane.text_processing import Text

with open("lyrics.txt") as f:
    lyrics = f.read()

audio_file = "Over The Next Rainbow [Q3HBLj90OTk].m4a"

lyrics_txt = Text.from_raw(lyrics)
waveform = load_audio(audio_file)
emission, token_spans = compute_alignments(waveform, lyrics_txt.transcript)


Using device=device(type='cpu')


In [5]:
from typing import Iterator

from pysubs2 import SSAEvent, SSAFile

from yohane.force_alignment import bundle
from yohane.text_processing import Line, Word


num_frames = emission.size(1)
ratio = waveform.size(1) / num_frames
sample_rate = bundle.sample_rate

subs = SSAFile()
event = SSAEvent()

lines_iter = iter(lyrics_txt.lines)
curr_line: Line
words_iter: Iterator[Word] = iter(())
curr_word: Word
syllables_iter: Iterator[str] = iter(())
curr_syllable: str

for i, spans in enumerate(token_spans):
    x0 = ratio * spans[0].start
    x1 = ratio * spans[-1].end

    t_start = x0 / sample_rate
    t_end = x1 / sample_rate

    next_t_start = None
    if i < len(token_spans) - 1:
        next_x0 = ratio * token_spans[i + 1][0].start
        next_t_start = next_x0 / sample_rate

    try:
        curr_syllable = next(syllables_iter)
    except StopIteration:
        try:
            # New word
            curr_word = next(words_iter)
            syllables_iter = iter(curr_word.syllables)
            curr_syllable = next(syllables_iter)

            event.text += " "
        except StopIteration:
            try:
                # New line
                curr_line = next(lines_iter)
                words_iter = iter(curr_line.words)
                curr_word = next(words_iter)
                syllables_iter = iter(curr_word.syllables)
                curr_syllable = next(syllables_iter)

                event.text = event.text.strip()
                if event.text:
                    subs.append(event)
                event = SSAEvent(int(t_start * 1000))
            except StopIteration:
                raise RuntimeError("???")

    k_start = t_start
    k_end = next_t_start if next_t_start is not None else t_end
    event.text += rf"{{\k{round((k_end - k_start) * 100)}}}{curr_syllable}"
    event.end = int(t_end * 1000)

print(subs.to_string("ass"))
# subs.save("kara.ass")


[Script Info]
; Script generated by pysubs2
; https://pypi.python.org/pypi/pysubs2
WrapStyle: 0
ScaledBorderAndShadow: yes
Collisions: Normal
ScriptType: v4.00+

[V4+ Styles]
Format: Name, Fontname, Fontsize, PrimaryColour, SecondaryColour, OutlineColour, BackColour, Bold, Italic, Underline, StrikeOut, ScaleX, ScaleY, Spacing, Angle, BorderStyle, Outline, Shadow, Alignment, MarginL, MarginR, MarginV, Encoding
Style: Default,Arial,20.0,&H00FFFFFF,&H000000FF,&H00000000,&H00000000,0,0,0,0,100.0,100.0,0.0,0.0,1,2.0,2.0,2,10,10,10,1

[Events]
Format: Layer, Start, End, Style, Name, MarginL, MarginR, MarginV, Effect, Text
Dialogue: 0,0:00:00.84,0:00:06.14,Default,,0,0,0,,{\k40}a{\k22}i{\k36}ta{\k70}ka{\k4}t{\k66}ta {\k44}to{\k38}o{\k24}i {\k34}ba{\k36}sho {\k10}ni {\k62}i{\k34}te {\k150}mo
Dialogue: 0,0:00:07.54,0:00:32.96,Default,,0,0,0,,{\k14}i{\k14}tsu{\k36}ka {\k22}ma{\k14}ta {\k40}ne {\k2}a{\k28}e{\k70}ru {\k2028}yo {\k832}ne
Dialogue: 0,0:00:38.54,0:00:43.44,Default,,0,0,0,,{\k22}yu{\k